This notebook shows how to use multi-armed bandits to build the agent to beat many (partially)deterministic agents.

For every public strategy (except random), it is easy to construct an algorithm that beats it. But you don't know in advance what method is used by the opponent.

The multi-armed bandit approach is a Reinforcement learning technic that allows us to select the most promising counter-strategy and use it to beat the opponent.

Multi-armed bandit is a widely used RL-algorithm because it is very balanced in terms of exploitation/exploration.

Algorithm logic:
- Create N agents with different logic
- At each step for each agent, generate a random number from B(a+1, b+1). B - beta-distribution, a - number of historical wins of this agent, b - number of this agent's historical losses.
- Select the agent with the largest generated number and use it to generate the next step.
- Repeat

In this notebook, I also show how to create the following types of agents:
- Mirror + shift. This class of agents takes the opponent's last step and return (step + shift) % 3. E.g., with shift=0 - it is a mirroring agent, with shift=1 agent that beats the opponent's previous hand.
- Self + shift. These agents add some shifts to their previous steps. With shift=0, it is a constant agent. With shift = 1 or 2, it just repeats all 3 possible steps in direct or reversed order.
- Popular beater - returns the step that beats the most popular action of the opponent so far.
- Anti-popular beater - return the step that beats the step that wins the most popular step )
- Transition matrix - computes the historical probability to go from one state to another (both for opponent and yourself) and returns the step that beats the most probable step of the opponent.
- Transition tensor - the same thing but considers both player's and opponent's previous steps. The last two agents have both deterministic and random versions and a decay option - higher weight for the more recent rounds.

In [1]:
%%writefile submission.py

import pandas as pd
import numpy as np

from numpy import random

import json


# base class for all agents, random agent
class agent():
    def initial_step(self):
        return np.random.randint(3)
    
    def history_step(self, history):
        return np.random.randint(3)
    
    def step(self, history):
        if len(history) == 0:
            return int(self.initial_step())
        else:
            return int(self.history_step(history))
    
# agent that returns (previousCompetitorStep + shift) % 3
class mirror_shift(agent):
    def __init__(self, shift=0):
        self.shift = shift
    
    def history_step(self, history):
        return (history[-1]['competitorStep'] + self.shift) % 3
    
    
# agent that returns (previousPlayerStep + shift) % 3
class self_shift(agent):
    def __init__(self, shift=0):
        self.shift = shift
    
    def history_step(self, history):
        return (history[-1]['step'] + self.shift) % 3    


# agent that beats the most popular step of competitor
class popular_beater(agent):
    def history_step(self, history):
        counts = np.bincount([x['competitorStep'] for x in history])
        return (int(np.argmax(counts)) + 1) % 3

    
# agent that beats the agent that beats the most popular step of competitor
class anti_popular_beater(agent):
    def history_step(self, history):
        counts = np.bincount([x['step'] for x in history])
        return (int(np.argmax(counts)) + 2) % 3
    
    
# simple transition matrix: previous step -> next step
class transition_matrix(agent):
    def __init__(self, deterministic = False, counter_strategy = False, init_value = 0.1, decay = 1):
        self.deterministic = deterministic
        self.counter_strategy = counter_strategy
        if counter_strategy:
            self.step_type = 'step' 
        else:
            self.step_type = 'competitorStep'
        self.init_value = init_value
        self.decay = decay
        
    def history_step(self, history):
        matrix = np.zeros((3,3)) + self.init_value
        for i in range(len(history) - 1):
            matrix = (matrix - self.init_value) / self.decay + self.init_value
            matrix[int(history[i][self.step_type]), int(history[i+1][self.step_type])] += 1

        if  self.deterministic:
            step = np.argmax(matrix[int(history[-1][self.step_type])])
        else:
            step = np.random.choice([0,1,2], p = matrix[int(history[-1][self.step_type])]/matrix[int(history[-1][self.step_type])].sum())
        
        if self.counter_strategy:
            # we predict our step using transition matrix (as competitor can do) and beat probable competitor step
            return (step + 2) % 3 
        else:
            # we just predict competitors step and beat it
            return (step + 1) % 3
    

# similar to the transition matrix but rely on both previous steps
class transition_tensor(agent):
    
    def __init__(self, deterministic = False, counter_strategy = False, init_value = 0.1, decay = 1):
        self.deterministic = deterministic
        self.counter_strategy = counter_strategy
        if counter_strategy:
            self.step_type1 = 'step' 
            self.step_type2 = 'competitorStep'
        else:
            self.step_type2 = 'step' 
            self.step_type1 = 'competitorStep'
        self.init_value = init_value
        self.decay = decay
        
    def history_step(self, history):
        matrix = np.zeros((3,3, 3)) + 0.1
        for i in range(len(history) - 1):
            matrix = (matrix - self.init_value) / self.decay + self.init_value
            matrix[int(history[i][self.step_type1]), int(history[i][self.step_type2]), int(history[i+1][self.step_type1])] += 1

        if  self.deterministic:
            step = np.argmax(matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])])
        else:
            step = np.random.choice([0,1,2], p = matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])]/matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])].sum())
        
        if self.counter_strategy:
            # we predict our step using transition matrix (as competitor can do) and beat probable competitor step
            return (step + 2) % 3 
        else:
            # we just predict competitors step and beat it
            return (step + 1) % 3

    
agents = {
    'mirror_0': mirror_shift(0),
    'mirror_1': mirror_shift(1),  
    'mirror_2': mirror_shift(2),
    'self_0': self_shift(0),
    'self_1': self_shift(1),  
    'self_2': self_shift(2),
    'popular_beater': popular_beater(),
    'anti_popular_beater': anti_popular_beater(),
    'random_transitison_matrix': transition_matrix(False, False),
    'determenistic_transitison_matrix': transition_matrix(True, False),
    'random_self_trans_matrix': transition_matrix(False, True),
    'determenistic_self_trans_matrix': transition_matrix(True, True),
    'random_transitison_tensor': transition_tensor(False, False),
    'determenistic_transitison_tensor': transition_tensor(True, False),
    'random_self_trans_tensor': transition_tensor(False, True),
    'determenistic_self_trans_tensor': transition_tensor(True, True),
    
    'random_transitison_matrix_decay': transition_matrix(False, False, decay = 1.05),
    'random_self_trans_matrix_decay': transition_matrix(False, True, decay = 1.05),
    'random_transitison_tensor_decay': transition_tensor(False, False, decay = 1.05),
    'random_self_trans_tensor_decay': transition_tensor(False, True, decay = 1.05),
    
    'determenistic_transitison_matrix_decay': transition_matrix(True, False, decay = 1.05),
    'determenistic_self_trans_matrix_decay': transition_matrix(True, True, decay = 1.05),
    'determenistic_transitison_tensor_decay': transition_tensor(True, False, decay = 1.05),
    'determenistic_self_trans_tensor_decay': transition_tensor(True, True, decay = 1.05),
}


score_sum = 0
flg = 0

def multi_armed_bandit_agent (observation, configuration):
    
    global score_sum, flg
    
    # bandits' params
    step_size = 2 # how much we increase a and b 
    decay_rate = 1.05 # how much do we decay old historical data
    
    # I don't see how to use any global variables, so will save everything to a CSV file
    # Using pandas for this is too much, but it can be useful later and it is convinient to analyze
    def save_history(history, file = 'history.csv'):
        pd.DataFrame(history).to_csv(file, index = False)

    def load_history(file = 'history.csv'):
        return pd.read_csv(file).to_dict('records')
    
    
    def log_step(step = None, history = None, agent = None, competitorStep = None):
        if step is None:
            step = np.random.randint(3)
        if history is None:
            history = []
        history.append({'step': step, 'competitorStep': competitorStep, 'agent': agent})
        save_history(history)
        return step
    
    def update_competitor_step(history, competitorStep):
        history[-1]['competitorStep'] = int(competitorStep)
        return history
        
    
    # load history
    if observation.step == 0:
        history = []
        bandit_state = {k:[1,1] for k in agents.keys()}
    else:
        history = update_competitor_step(load_history(), observation.lastOpponentAction)
        
        if history[-1]['step']==1 and history[-1]['competitorStep']==0:
            score_sum += 1
        elif history[-1]['step']==2 and history[-1]['competitorStep']==1:
            score_sum += 1
        elif history[-1]['step']==0 and history[-1]['competitorStep']==2:   
            score_sum += 1
        elif history[-1]['step']==0 and history[-1]['competitorStep']==1:
            score_sum += -1
        elif history[-1]['step']==1 and history[-1]['competitorStep']==2:
            score_sum += -1
        elif history[-1]['step']==2 and history[-1]['competitorStep']==0:
            score_sum += -1
        
        # load the state of the bandit
        with open('bandit.json') as json_file:
            bandit_state = json.load(json_file)
        
        # updating bandit_state using the result of the previous step
        # we can update all states even those that were not used
        for name, agent in agents.items():
            agent_step = agent.step(history[:-1])
            bandit_state[name][1] = (bandit_state[name][1] - 1) / decay_rate + 1
            bandit_state[name][0] = (bandit_state[name][0] - 1) / decay_rate + 1
            
            if (history[-1]['competitorStep'] - agent_step) % 3 == 1:
                bandit_state[name][1] += step_size
            elif (history[-1]['competitorStep'] - agent_step) % 3 == 2:
                bandit_state[name][0] += step_size
            else:
                bandit_state[name][0] += step_size/2
                bandit_state[name][1] += step_size/2
            
    with open('bandit.json', 'w') as outfile:
        json.dump(bandit_state, outfile)
    
    #RPS: Opponent Transition Matrix
    my_list = [0] * 34 + [1] * 33 + [2] * 33
    result = int(random.choice(my_list))
    

    # generate random number from Beta distribution for each agent and select the most lucky one
    best_proba = -1
    best_agent = None
    for k in bandit_state.keys():
        proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
        if proba > best_proba:
            best_proba = proba
            best_agent = k
        
    step = agents[best_agent].step(history)
    
    if observation.step>500 and score_sum<-20:
        flg = 1
        return result
    elif flg == 1:
        return result
    else:
        return log_step(step, history, best_agent)

Writing submission.py


Let's beat the mirroring agent using our solutioin:

In [2]:
%%writefile copy_opponent.py
    
def copy_opponent_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return 0

Writing copy_opponent.py


In [3]:
from kaggle_environments import evaluate, make, utils
env = make("rps", debug=True)

Failed: football: No module named 'gfootball'


In [4]:
env.reset()
env.run(["submission.py", "copy_opponent.py"])
env.render(mode="ipython", width=800, height=700)

Let's look at the final state of the bandit.

In [5]:
import json

In [6]:
with open('bandit.json') as json_file:
    bandit_state = json.load(json_file)
bandit_state

{'mirror_0': [1.0017964255626708, 42.998203573600414],
 'mirror_1': [22.00044990754049, 21.999550091622613],
 'mirror_2': [42.99775366566242, 1.0022463335006844],
 'self_0': [21.999999999602473, 21.99999999956063],
 'self_1': [42.99999999912126, 1.0000000000418432],
 'self_2': [1.0000000000209215, 42.99999999914218],
 'popular_beater': [22.349693950069586, 21.65030604909352],
 'anti_popular_beater': [21.316346157687374, 22.683653841475735],
 'random_transitison_matrix': [42.21348200579261, 1.7865179933704867],
 'determenistic_transitison_matrix': [42.99775366564645, 1.0022463335166543],
 'random_self_trans_matrix': [22.059634519627934, 21.94036547953517],
 'determenistic_self_trans_matrix': [21.999999999374232, 21.999999999788876],
 'random_transitison_tensor': [42.99567144584006, 1.0043285533230513],
 'determenistic_transitison_tensor': [42.999999999013454, 1.000000000149647],
 'random_self_trans_tensor': [21.719288382288752, 22.280711616874353],
 'determenistic_self_trans_tensor': [2

We can see that self_1 was applied much often than other strategies, and it is a clear winner. It is a counter-strategy for the simple mirroring strategy. That is why a multi-armed bandit algorithm used it much more often than any other agent.

Let's now create an agent acting as the self_1 agent (counter strategy for the mirroring algorithm https://www.kaggle.com/ilialar/beating-mirror-agent) and compare it to our bandit. 

In [7]:
%%writefile anti_mirror.py

import pandas as pd
    
def anti_copy_opponent_agent (observation, configuration):
    
    # I don't see how to use any global variables, so will save everything to a CSV file
    # Using pandas for this is too much, but it can be useful later
    def save_history(history, file = 'history2.csv'):
        pd.DataFrame(history).to_csv(file, index = False)

    def load_history(file = 'history2.csv'):
        return pd.read_csv(file).to_dict('records')
    
    if observation.step == 0:
        history = [{'step': 1, 'competitorStep': None}]
        save_history(history)
        return 1
    else:
        history = load_history()
        history[-1]['competitorStep'] = observation.lastOpponentAction
        step = (history[-1]['step'] + 1) % 3
        history.append({'step': step, 'competitorStep': None})
        save_history(history)
        return step

Writing anti_mirror.py


In [8]:
env.reset()
env.run(["submission.py", "anti_mirror.py"])
env.render(mode="ipython", width=800, height=700)

In [9]:
with open('bandit.json') as json_file:
    bandit_state = json.load(json_file)
bandit_state

{'mirror_0': [1.0000000000219675, 42.999999999099295],
 'mirror_1': [21.9999999995826, 21.999999999538662],
 'mirror_2': [42.99999999912126, 1.0],
 'self_0': [22.000139170132318, 21.999860828988947],
 'self_1': [42.99972165761743, 1.0002783415038368],
 'self_2': [1.0001391709760834, 42.99986082814519],
 'popular_beater': [14.655828707111352, 29.34417129200991],
 'anti_popular_beater': [22.349722442037294, 21.650277557083978],
 'random_transitison_matrix': [42.99999999847554, 1.0000000006457272],
 'determenistic_transitison_matrix': [42.999999999027736, 1.0000000000935296],
 'random_self_trans_matrix': [22.154958535939237, 21.84504146318202],
 'determenistic_self_trans_matrix': [22.000139170130147, 21.99986082899111],
 'random_transitison_tensor': [42.999999998445524, 1.000000000675739],
 'determenistic_transitison_tensor': [42.99999999892208, 1.0000000001991747],
 'random_self_trans_tensor': [21.955490683800186, 22.044509315321076],
 'determenistic_self_trans_tensor': [21.9999999995559

Our bandit is also the winner, but it mostly used the mirror_2 strategy that is the optimal counter-strategy to the self_1 agent.

You can add your algorithms to the `agents` dict. If one of your agents works well against the current opponent, a multi-armed bandit will find it out and use it to win.